# Libraries

In [ ]:
# google drive access
from google.colab import drive
drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import torchvision
from sklearn.metrics import accuracy_score

# install and import the torchinfo library
!pip install torchinfo
from torchinfo import summary

# Model

In [ ]:
class CCM(nn.Module):
    """
    Channel wise calibration model:
        y = x * Sigmoid(BN(Group_Conv(GAP(x))))
    """
    def __init__(self, in_channel, out_channel):
        super(CCM, self).__init__()
        self.adapool = nn.AdaptiveAvgPool2d(1)
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, 
            stride=1, padding=0, groups=in_channel, bias=False)
        self.bn = nn.BatchNorm2d(out_channel)
        self.sig = nn.Sigmoid()

    def forward(self, x):
        out = self.adapool(x)
        out = self.conv(out)
        out = self.bn(out)
        out = self.sig(out)
        out = x * out
        return out


class SCM(nn.Module):
    """
    Spatial Calibration Module:
        y = x + Group_Conv(x)
    """
    def __init__(self, channel):
        super(SCM, self).__init__()
        self.conv = nn.Conv2d(channel, channel, kernel_size=3,
            stride=1, padding=1, groups=channel, bias=False)

    def forward(self, x):
        out = self.conv(x)
        out = x + out
        return out


class Shortcut(nn.Module):
    """
    Modified skip connections in ResNet 
    with calibration modules.
    """
    def __init__(self, in_channel, out_channel, stride=1):
        super(Shortcut, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, kernel_size=1, 
            stride=stride, bias=False)
        self.scm = SCM(out_channel)
        self.bn = nn.BatchNorm2d(out_channel)
        self.ccm = CCM(out_channel, out_channel)

    def forward(self, x):
        out = self.conv(x)
        out = self.scm(out)
        out = self.bn(out)
        out = self.ccm(out)
        return out


class BasicBlock(nn.Module):
    expansion = 1
    """
    Convolutional blocks of ResNet with 
    skip connections and calibration modules.
    """
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, 
            stride=stride, padding=1, bias=False)
        # SCM and CCM for conv1
        self.scm1 = SCM(planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.ccm1 = CCM(planes, planes)

        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, 
            stride=1, padding=1, bias=False)
        # SCM and CCM for conv2
        self.scm2 = SCM(planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.ccm2 = CCM(planes, planes)

        self.shortcut = None
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = Shortcut(in_planes, self.expansion*planes, stride)

    def forward(self, x):
        # conv1 forward pass
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # conv2 forward pass
        out = self.conv2(out)
        out = self.scm2(out)
        out = self.bn2(out)
        out = self.ccm2(out)
        if self.shortcut is not None:
            out += self.shortcut(x)
        else:
            out += x
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64
        # input convolutions
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.scm1 = SCM(64)
        self.bn1 = nn.BatchNorm2d(64)
        self.ccm1 = CCM(64, 64)
        # residual blocks
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        # classifier head
        self.classifier = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        # forward pass of input conv
        out = self.conv1(x)
        out = self.scm1(out)
        out = self.bn1(out)
        out = self.ccm1(out)
        out = F.relu(out)
        # forward pass of residual blocks
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        # forward pass of classifier head
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out


def get_resnet18(num_classes=10):
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes=num_classes)

# Utils

In [ ]:
def freeze_noncalibration_layers(model):
    for child in model.children():
        # Base model convs are frozen
        # print('Child: ', type(child))
        if type(child) == nn.Conv2d:
            froze_this_layer(child, tab='')
        # Search for basic blocks in Sequentials
        elif type(child) == nn.Sequential:
            for child2 in child.children():
                # print('\t Child2: ', type(child2))
                if type(child2) == nn.Conv2d:
                    froze_this_layer(child2, tab='\t ')
                # Search for Shortcuts in Basic Blocks
                elif type(child2) == BasicBlock:
                    for child3 in child2.children():
                        # print('\t\t Child3: ', type(child3))
                        # Base model convs are frozen
                        if type(child3) == nn.Conv2d:
                            froze_this_layer(child3, tab='\t\t ')
                        elif type(child3) == Shortcut:
                            # Base model convs are frozen
                            for child4 in child3.children():
                                # print('\t\t\t Child4: ', type(child4))
                                if type(child4) == nn.Conv2d:
                                    froze_this_layer(child4, tab='\t\t\t ')

def froze_this_layer(layer, tab=''):
    for param in layer.parameters():
        # print(tab + '********* Conv2d frozen *********')
        param.requires_grad = False


def accuracy(y, y_hat):
    y_hat = (np.squeeze(y_hat) >= 0.5)
    y = np.squeeze(y)
    acc = accuracy_score(y, y_hat)
    return acc

# Dataset


In [ ]:
def get_dataloader(task_no=0, subset='train', batch_size=128, val_ratio=0.2, vis=False):
    if subset == 'test':
        raw_data = torchvision.datasets.SVHN(root='./data', split='test', download=True)
        shuffle = False
        bsize = len(raw_data)
    else:
        raw_data = torchvision.datasets.SVHN(root='./data', split='train', download=True)
        val_len = int(len(raw_data) * val_ratio)
        train_len = int(len(raw_data) - val_len)
        raw_train, raw_val = random_split(raw_data, [train_len, val_len], 
                                          generator=torch.Generator().manual_seed(42))
        if subset == 'train':
            raw_data = raw_train
            shuffle = True
            bsize = batch_size
        elif subset == 'val':
            raw_data = raw_val
            shuffle = False
            bsize = len(raw_data)

    if vis:
        classes = list(map(lambda x: 'No'+str(x), np.unique(raw_data.labels).tolist()))
        num_classes = 10
        samples_per_class = 7
        visualize_samples(raw_data, classes, num_classes, samples_per_class)
    
    # Transforms: conversion to tensor and normalization
    mean = [0.43768206, 0.44376972, 0.47280434] 
    std = [0.19803014, 0.20101564, 0.19703615]
    transforms = torchvision.transforms.Compose(
            [torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean, std)])
    
    task_data = CustomSVHN(raw_data, task_no, transforms)
    print('Num. samples ({}) in task dataset: {}'.format(subset, len(task_data)))
    dataloader = DataLoader(task_data, batch_size=bsize, 
                            shuffle=shuffle, num_workers=2)
    return dataloader


def visualize_samples(dataset, classes, num_classes, samples_per_class):
    for y, cls in enumerate(classes):
        idxs = np.flatnonzero(dataset.labels == y)
        idxs = np.random.choice(idxs, samples_per_class, replace=False)
        for i, idx in enumerate(idxs):
            plt_idx = i * num_classes + y + 1
            plt.subplot(samples_per_class, num_classes, plt_idx)
            img, label = dataset[idx]
            plt.imshow(img)
            plt.axis('off')
            if i == 0:
                plt.title(cls)
    plt.show()


class CustomSVHN(Dataset):
    def __init__(self, dataset, task_no=0, transforms=None):
        super(CustomSVHN, self).__init__()
        self.dataset = dataset
        self.task_no = task_no
        self.transforms = transforms
        self.classes = [2*task_no, 2*task_no+1]
        self.idxs = [idx for idx in range(len(self.dataset)) 
                    if (self.dataset[idx][1] in self.classes)]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        img, y = self.dataset[self.idxs[idx]]
        y -= 2*self.task_no
        # convert labels to torch tensor
        if not isinstance(y, np.ndarray):
            y = np.array([y])
            y = torch.from_numpy(y).float()
        # apply input image transformations
        if self.transforms:
            img = self.transforms(img)
        return img, y

# Main

In [ ]:
if torch.cuda.is_available():
  print("Cuda (GPU support) is available and enabled!")
  device = torch.device("cuda")
else:
  print("Cuda (GPU support) is not available")
  device = torch.device("cpu")

"""
model = ResNet(BasicBlock, [2,2,2,2], num_classes=1)
summary(model, input_size=(16, 3, 32, 32))
"""

Cuda (GPU support) is available and enabled!


'\nmodel = ResNet(BasicBlock, [2,2,2,2], num_classes=1)\nsummary(model, input_size=(16, 3, 32, 32))\n'

In [ ]:
def train(verbose=True):
    # create ResNet18 model
    model = ResNet(BasicBlock, [2,2,2,2], num_classes=1)
    model = model.to(device)

    # tensorboard logger
    writer = SummaryWriter(log_dir='runs/SVHN_experiment')
    model_path = './models/SVHN_experiment/'
    # training parameters
    batch_size = 128
    lr = 1e-2
    epochs_per_task = 20

    for task_no in range(5):
        task_tag = 'task_' + str(task_no)
        # freeze layers other than CCM and SCM,
        # add new classifier head for next task
        if task_no > 0:
            print('************* Re-calibrating model parameters *************')
            model = model.to('cpu')
            freeze_noncalibration_layers(model)
            model.classifier = nn.Linear(512*BasicBlock.expansion, 1)
            model = model.to(device)

        # check layer freezing
        num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print('************* Task: {}, Number of Trainable Parameters: {} *************'.format(task_no, num_trainable_params))

        # get dataset for task no
        task_train_loader = get_dataloader(task_no=task_no, subset='train', batch_size=batch_size)
        task_val_loader = get_dataloader(task_no=task_no, subset='val')

        # loss, optimizer and lr-scheduler are identical for each task
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.SGD(model.parameters(), lr, momentum=0.9)
        ###scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[50, 100, 125], gamma=0.1)

        # training iterations
        tr_loss_hist, val_loss_hist, val_acc_hist = [], [], []
        num_iter = 0
        for e in range(epochs_per_task):
            # put model to training mode after each eval
            model.train()
            # training for an epoch
            for _, data in tqdm(enumerate(task_train_loader)):
                # get the data
                x, y = data
                x, y = x.to(device), y.to(device)

                # clear gradients
                optimizer.zero_grad()

                # calculate loss
                yhat = model(x)
                loss = criterion(yhat, y)

                # backprop
                loss.backward()

                # update parameters
                optimizer.step()

                # store training loss
                tr_loss_hist.append(loss.item())
                # save training loss per minibatch iterations
                writer.add_scalar('tr_loss/'+task_tag, loss.item(), num_iter)
                num_iter += 1

            # get average training loss for last 5 iterations
            tr_loss = np.sum(tr_loss_hist[:-6:-1])/5
            # update learning-rate
            ###scheduler.step()
            
            # validation after each epoch
            model.eval()
            with torch.no_grad():
                for i, data in enumerate(task_val_loader):
                    x, y = data
                    x, y = x.to(device), y.to(device)
                    yhat = model(x)
                # compute validation metrics
                val_loss = criterion(yhat, y) .item()
                val_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())

            # save validation scores
            val_loss_hist.append(val_loss)
            val_acc_hist.append(val_acc)
            # save to tensorboard
            writer.add_scalar('val_loss/'+task_tag, val_loss, e)
            writer.add_scalar('val_acc/'+task_tag, val_acc, e)
            # display training info
            if verbose:
                print('Epoch {}/{} || tr_loss:{} || val_loss:{} || val_acc:{}'.format(
                    e+1, epochs_per_task, tr_loss, val_loss, val_acc))
                
        # save the model for current task
        print('************* Saving model for Task-{} *************'.format(task_no))
        model_filename = 'svhn_model_task_' + str(task_no) + '.pt'
        torch.save(model, model_path + model_filename)
        print('************* End of Task-{} training *************'.format(task_no))

In [ ]:
train(verbose=True)

************* Task: 0, Number of Trainable Parameters: 11226945 *************
Using downloaded and verified file: ./data/train_32x32.mat
Num. samples (train) in task dataset: 15053
Using downloaded and verified file: ./data/train_32x32.mat
Num. samples (val) in task dataset: 3756


118it [00:24,  4.86it/s]


Epoch 1/20 || tr_loss:0.3624118506908417 || val_loss:0.37616023421287537 || val_acc:0.8362619808306709


118it [00:24,  4.79it/s]


Epoch 2/20 || tr_loss:0.1727960154414177 || val_loss:0.16503840684890747 || val_acc:0.9345047923322684


...

Epoch 18/20 || tr_loss:0.1892694890499115 || val_loss:0.20302169024944305 || val_acc:0.9118678017025539


61it [00:09,  6.72it/s]


Epoch 19/20 || tr_loss:0.15027111917734146 || val_loss:0.19933883845806122 || val_acc:0.9178768152228343


61it [00:09,  6.68it/s]


Epoch 20/20 || tr_loss:0.19144816994667052 || val_loss:0.202229306101799 || val_acc:0.9108662994491737
************* Saving model for Task-4 *************
************* End of Task-4 training *************


In [ ]:
def test():
    model_path = './models/SVHN_experiment/'
    for task_no in range(5):
        # load models per task
        series_num = 'svhn_model_task_' + str(task_no) + '.pt'
        model = torch.load(model_path + series_num)
        model = model.to(device)
        model.eval()
        # get task specific test data
        test_loader = get_dataloader(subset='test', task_no=task_no)
        criterion = nn.BCEWithLogitsLoss()

        with torch.no_grad():
            for _, data in enumerate(test_loader):
                x, y = data
                x, y = x.to(device), y.to(device)
                yhat = model(x)

                test_loss = criterion(yhat, y).item()
                test_acc = accuracy(y.to('cpu').numpy(), yhat.to('cpu').numpy())
            print('Task-{} || \t test_loss:{} || \t test_acc:{}'.format(task_no, test_loss, test_acc))

In [ ]:
test()

Using downloaded and verified file: ./data/test_32x32.mat
Num. samples (test) in task dataset: 6843
Task-0 || 	 test_loss:0.12139896303415298 || 	 test_acc:0.9720882653806809
Using downloaded and verified file: ./data/test_32x32.mat
Num. samples (test) in task dataset: 7031
Task-1 || 	 test_loss:0.18492311239242554 || 	 test_acc:0.9320153605461527
Using downloaded and verified file: ./data/test_32x32.mat
Num. samples (test) in task dataset: 4907
Task-2 || 	 test_loss:0.11212003231048584 || 	 test_acc:0.9543509272467903
Using downloaded and verified file: ./data/test_32x32.mat
Num. samples (test) in task dataset: 3996
Task-3 || 	 test_loss:0.07426847517490387 || 	 test_acc:0.9724724724724725
Using downloaded and verified file: ./data/test_32x32.mat
Num. samples (test) in task dataset: 3255
Task-4 || 	 test_loss:0.19009599089622498 || 	 test_acc:0.9265745007680491
